# **Маркетинг: определение потенциального покупателя**

# TODO  сдать до 9 марта--------------------------------------------------

# **1. Описание проекта**

Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи. 

Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

## **1.1. Цель**
Предсказать вероятность покупки в течение 90 дней


## **1.2. Задачи**

● Изучить данные

● Разработать полезные признаки

● Создать модель для классификации пользователей

● Улучшить модель и максимизировать метрику roc_auc

● Выполнить тестирование

## **1.3. Описание данных**

### **apparel-purchases** - история покупок

Данные о покупках клиентов по дням и по товарам. В каждой записи покупка
определенного товара, его цена, количество штук.
В таблице есть списки идентификаторов, к каким категориям относится товар. Часто
это вложенные категории (например автотовары-аксессуары-освежители), но также может
включать в начале списка маркер распродажи или маркер женщинам/мужчинам.
Нумерация категорий сквозная для всех уровней, то есть 44 на второй позиции списка
или на третьей – это одна и та же категория. Иногда дерево категорий обновляется, поэтому
могут меняться вложенности, например ['4', '28', '44', '1594'] или ['4', '44', '1594']. Как
обработать такие случаи – можете предлагать свои варианты решения.


- `client_id` - идентификатор пользователя
- `quantity` - количество товаров в заказе
- `price` - цена товара
- `category_ids` - вложенные категории, к которым отнсится товар
- `date` дата - покупки
- `message_id` - идентификатор сообщения из рассылки

### **apparel-messages** - история рекламных рассылок

Рассылки, которые были отправлены клиентам из таблицы покупок.


- `bulk_campaign_id` - идентификатор рекламной кампании (рассылки)
- `client_id` - идентификатор пользователя
- `message_id` - идентификатор сообщений
- `event` - тип действия с сообщением (отправлено, открыто, покупка…)
- `channel` - канал рассылки
- `date` - дата рассылки
- `created_at` - точное время и дата создания сообщения


### **apparel-target_binary** - покупка в течение следующих 90 дней


- `client_id` - идентификатор пользователя
- `target` - целевой признак - клиент совершил покупку в целевом периоде

### **full_campaign_daily_event** - агрегация общей базы рассылок по дням и типам событий

- `date` - дата
- `bulk_campaign_id` - идентификатор рассылки
- `count_event` - общее количество каждого события event (все типы событий event)
- `nunique_event` - количество уникальных client_id в каждом событии (все типы событий event)


### **full_campaign_daily_event_channel** - агрегация по дням с учетом событий и каналов рассылки

- `date` - дата
- `bulk_campaign_id` - идентификатор рассылки
- `count_event` - общее количество каждого события по каналам *
- `nunique_event` - количество уникальных client_id по событиям и каналам *

* в именах колонок есть все типы событий event и каналов рассылки channel

нельзя суммировать по колонкам nunique, потому что это уникальные клиенты в пределах дня, нет данных, повторяются ли они в другие дни.

## **1.4. Требования к оформлению**

Репозиторий на гитхабе:

- тетрадь jupyter notebook с описанием, подготовкой признаков, обучением модели и тестированием
- описание проекта и инструкция по использованию в файле README.md
- список зависимостей в файле requirements.txt



## **1.5. Стэк**

● python
● pandas
● sklearn



# **2. Настройка рабочего пространства**

## **2.1. Импорт библиотек и настройка рабочего пространства.**

In [1]:
from IPython.display import display, HTML 
import warnings

import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Any, Callable, Dict, Optional, Union

from phik import phik_matrix

import lightgbm as lgb
from sklearn.linear_model import Ridge

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

from wordcloud import WordCloud

In [2]:
warnings.filterwarnings('ignore') # чтобы не было красный полей с предупреждениями об устаревших библиотеках
# %matplotlib inline
plt.ion() # принудительное отображение графиков matplotlib в VS Code
pd.set_option("display.max_columns", None) # чтобы сам df был пошире
pd.set_option('display.max_colwidth', None) # чтобы df колонки были пошире
pd.set_option('display.float_format', '{:.3f}'.format) # округление чисел в df, чтобы числа не печатал экспоненциально
pd.options.display.expand_frame_repr = False # для принта чтобы колонки не переносил рабоатет тольок в vs code, in jupyter notebook получается каша

## **2.2. Загрузка данных**

In [6]:
try:
    messages_df = pd.read_csv('datasets/apparel-messages.csv')
    purchases_df = pd.read_csv('datasets/apparel-purchases.csv')
    target_df = pd.read_csv('datasets/apparel-target_binary.csv')
    event_cahnnel_df = pd.read_csv('datasets/full_campaign_daily_event_channel.csv')
    event_df = pd.read_csv('datasets/full_campaign_daily_event.csv')
    print("Данные загружены с домашнего компьютера")
except (FileNotFoundError, OSError):
    # Альтернативный путь для запуска из Интернета
    messages_df = pd.read_csv('datasets/apparel-messages.csv')
    purchases_df = pd.read_csv('datasets/apparel-purchases.csv')
    target_df = pd.read_csv('datasets/apparel-target_binary.csv')
    event_cahnnel_df = pd.read_csv('datasets/full_campaign_daily_event_channel.csv')
    event_df = pd.read_csv('datasets/full_campaign_daily_event.csv')
    print("Данные загружены из Интернета")

Данные загружены с домашнего компьютера


In [9]:
df_list = [messages_df, purchases_df, target_df, event_cahnnel_df, event_df]
for df in df_list:
    display(df.head())

,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
0,4439,1515915625626736623,1515915625626736623-4439-6283415ac07ea,open,email,2022-05-19,2022-05-19 00:14:20
1,4439,1515915625490086521,1515915625490086521-4439-62834150016dd,open,email,2022-05-19,2022-05-19 00:39:34
2,4439,1515915625553578558,1515915625553578558-4439-6283415b36b4f,open,email,2022-05-19,2022-05-19 00:51:49
3,4439,1515915625553578558,1515915625553578558-4439-6283415b36b4f,click,email,2022-05-19,2022-05-19 00:52:20
4,4439,1515915625471518311,1515915625471518311-4439-628341570c133,open,email,2022-05-19,2022-05-19 00:56:52


,client_id,quantity,price,category_ids,date,message_id
0,1515915625468169594,1,1999.000,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
1,1515915625468169594,1,2499.000,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
2,1515915625471138230,1,6499.000,"['4', '28', '57', '431']",2022-05-16,1515915625471138230-4437-6282242f27843
3,1515915625471138230,1,4999.000,"['4', '28', '244', '432']",2022-05-16,1515915625471138230-4437-6282242f27843
4,1515915625471138230,1,4999.000,"['4', '28', '49', '413']",2022-05-16,1515915625471138230-4437-6282242f27843


,client_id,target
0,1515915625468060902,0
1,1515915625468061003,1
2,1515915625468061099,0
3,1515915625468061100,0
4,1515915625468061170,0


,date,bulk_campaign_id,count_click_email,count_click_mobile_push,count_open_email,count_open_mobile_push,count_purchase_email,count_purchase_mobile_push,count_soft_bounce_email,count_subscribe_email,count_unsubscribe_email,nunique_click_email,nunique_click_mobile_push,nunique_open_email,nunique_open_mobile_push,nunique_purchase_email,nunique_purchase_mobile_push,nunique_soft_bounce_email,nunique_subscribe_email,nunique_unsubscribe_email,count_hard_bounce_mobile_push,count_send_mobile_push,nunique_hard_bounce_mobile_push,nunique_send_mobile_push,count_hard_bounce_email,count_hbq_spam_email,count_send_email,nunique_hard_bounce_email,nunique_hbq_spam_email,nunique_send_email,count_soft_bounce_mobile_push,nunique_soft_bounce_mobile_push,count_complain_email,nunique_complain_email,count_close_mobile_push,nunique_close_mobile_push
0,2022-05-19,563,0,0,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,date,bulk_campaign_id,count_click,count_complain,count_hard_bounce,count_open,count_purchase,count_send,count_soft_bounce,count_subscribe,count_unsubscribe,nunique_click,nunique_complain,nunique_hard_bounce,nunique_open,nunique_purchase,nunique_send,nunique_soft_bounce,nunique_subscribe,nunique_unsubscribe,count_hbq_spam,nunique_hbq_spam,count_close,nunique_close
0,2022-05-19,563,0,0,0,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


- 3) Для того, чтобы **выполнить предобработку**, необходимо сначала отделить тестовую выборку;
- 2) Для того, чтобы  **отделить тестовую выборку**, необходимо объединить разрозненные таблицы;
- 1) Для того, чтобы  **объединить таблицы**, необходимо понять как это сделать так, чтоб объединение произошло корректно.

Для начала обратим внимание, что наш целевой признак (совершил покупку или нет) в `apparel-target_binary` == 1/0 для каждого `client_id`, а каждый `client_id` уникальный в `target_df` (одна строка == один клиент и значение 1/0 к нему), а значит, надо другие исходные таблицы перед объединением группировать по `client_id`, чтобы там `client_id` оставался уникальным.

Поэтому мы предлагаем следующую тактику объединения:
1) Агрегировать покупки по client_id: сколько покупок, суммарный чек, сколько разных категорий, давность последней покупки и т.п.
2) Агрегировать сообщения по client_id: сколько сообщений всего, сколько по каждому каналу, сколько открытий, сколько кликов и т.д.
Получить две таблицы с одной строкой на client_id, например purchases_agg и messages_agg.
3) Сделать финальный merge на уровне клиента

In [ ]:
purchases_agg = purchases_df.groupby('client_id').agg(
    total_purchases=('quantity', 'sum'),
    total_amount=('price', 'sum'),
    total_categories=('category_ids', 'nunique'),
    last_purchase_date=('date', 'max')
).reset_index()

messages_agg = messages_df.groupby('client_id').agg(
    total_messages=('message_id', 'count'),
    total_opens=('event', lambda x: (x == 'open').sum()),
    total_clicks=('event', lambda x: (x == 'click').sum()),
    total_purchases=('event', lambda x: (x == 'purchase').sum())
).reset_index()


In [ ]:
   X = (purchases_agg
        .merge(messages_agg, on='client_id', how='outer')
        .merge(target_df,   on='client_id', how='inner'))  # обычно inner по таргету

## **2.3. Сокращение размерности**

Для ускорения предварительной обработки и обучения сократим датасет

In [ ]:
# df = df[:100_000]

## **2.4. Словарь датафреймов для итераций**

In [ ]:
df_dict = {
            'train_df': train_df, 
            'test_df': test_df, 
            'val_df': val_df
            }

## **2.5. Деление данных на выборки**

In [ ]:
# Разделение всего датафрейма
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Train/Val split
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

total = len(df)

print(f'Train: {train_df.shape[0]:>6} строк ({train_df.shape[0]/total*100:>5.1f}%)')
print(f'Valid: {val_df.shape[0]:>6} строк ({val_df.shape[0]/total*100:>5.1f}%)')
print(f'Test:  {test_df.shape[0]:>6} строк ({test_df.shape[0]/total*100:>5.1f}%)')

Для удобства итераций и предобработки создадим словарь датафреймов.

## **2.6. Переименуем столбцы**

In [ ]:
for df in df_dict.values():
    df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower().replace(' ', '_').replace('-', '_') for col in df.columns]
    df.columns

# **3. EDA: исследовательский анализ данных**

# **3.1. Оценка качества представленных данных**

In [ ]:
df.info()   


In [ ]:
df

In [ ]:
df.describe()

## **3.2. Вывод о качестве данных**

## **3.3. Создание pipeline для предобработки**

Произведем предобработку данных на классах в PipeLine. Для этого создадим следующие классы:

- `DecimalPointChanger` - проверяет каждое значение столбца на наличие правильного разделителя дроби, в случае если будет найдена запятая - заменит ее на точку;
- `OutlierRemover` - удалит выбросы;
- `ImplicitDuplicatesViewer` - отобразит список уникальных нечисловых значений каждого столбца, что должно помочь опредлелить неявные дубликаты в столбцах;
- `DuplicateRemover` - удалит явные дубликаты;
- `MissingValueHandler` - обрабатывает пропуски на основе выбранной стратегии, по умолчанию, удаляет всю строку, если есть в ней пропуск;
- `ColumnRemover` - удаляет лишний столбцы из датафрейма;
- `FloatToIntChanger` - преобразует дробное число в целочисленное.

Эти классы мы передадим в класс `EDAPreprocessor`, который станет основной состаляющей пайплайна EDA_Preprocessor_pipline, который будет производить предобработку данных. Зпуск пайплайна буддет вызываться функией  `run_preprocessor()`.

In [ ]:
class MistakeCorrector(BaseEstimator, TransformerMixin):
    """Класс для исправления ошибок в данных."""
    
    def __init__(
            self, 
            columns: List[str], 
            values_dict: Optional[Dict[Any, Any]] = None, 
            func: Optional[Callable] = None,
            strategy: str = 'auto',
            skip_on_test: bool = False
            ):
        
        if not columns:
            raise ValueError("Параметр 'columns' не может быть пустым")
            
        if strategy not in ['dict', 'func', 'auto']:
            raise ValueError("strategy должен быть 'dict', 'func' или 'auto'")

        self.values_dict = values_dict or {}
        self.columns = columns
        self.func = func
        self.strategy = strategy
        self.skip_on_test = skip_on_test
        self.fill_values = {}
    
    def fit(self, X: pd.DataFrame, y=None, **kwargs):
        """Вычисляет значения для замены на train"""
        if self.strategy == 'auto':
            for col in self.columns:
                for invalid_val, method in self.values_dict.items():
                    # Если метод - это статистика
                    if method in ['median', 'mean', 'mode']:
                        valid_data = X[col][X[col] != invalid_val]
                        
                        if method == 'median':
                            self.fill_values[(col, invalid_val)] = valid_data.median()
                        elif method == 'mean':
                            self.fill_values[(col, invalid_val)] = valid_data.mean()
                        elif method == 'mode':
                            self.fill_values[(col, invalid_val)] = valid_data.mode()[0]
                    else:
                        # Если метод - это конкретное значение (например, 'petrol')
                        self.fill_values[(col, invalid_val)] = method
        return self
    
    def transform(self, X: pd.DataFrame, y=None, name=None) -> pd.DataFrame:
        """Применяет исправления"""
        df = X.copy()
        
        for col in self.columns:
            if col not in df.columns:
                continue

            if self.strategy == 'auto':
                for invalid_val, method in self.values_dict.items():
                    mask = df[col] == invalid_val
                    if mask.any():
                        fill_val = self.fill_values[(col, invalid_val)]
                        
                        # Определяем тип замены для вывода
                        if method in ['median', 'mean', 'mode']:
                            print(f"- Заменено {mask.sum()} значений '{invalid_val}' в '{col}' на {fill_val} ({method})")
                        else:
                            print(f"- Заменено {mask.sum()} значений '{invalid_val}' в '{col}' на '{fill_val}'")
                        
                        df.loc[mask, col] = fill_val
                        
            elif self.strategy == 'func' and self.func:
                df[col] = df[col].apply(self.func)
                
            elif self.strategy == 'dict' and self.values_dict:
                df[col] = df[col].replace(self.values_dict)
        
        return df

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X, **fit_params)

In [ ]:
class DecimalPointChanger(BaseEstimator, TransformerMixin):
    """Класс для замены разделителя дроби в строковых столбцах для обеих выборок"""

    def __init__(self, columns: List[str], skip_on_test=False):
        """Инициализация заменщика дроби в строковых столбцах"""
        
        self.columns = columns  # список столбцов, в которых нужно заменить разделитель дроби
        self.skip_on_test = skip_on_test
    
    def fit_transform(self, X: Union[pd.DataFrame, np.array], y: None = None, **fit_params) -> np.ndarray:
        
        """Непосредственно заменяет разделитель дроби запятую на точку"""

        print('- Определяю необходимость замены запятой на точку')
        
        df = X.copy()

        # Если columns не указаны, обрабатываем все столбцы
        cols_to_process = self.columns if self.columns else df.columns

        for col in cols_to_process:
            if col in df.columns:
                # Проверяем, есть ли запятые в столбце
                if df[col].dtype == 'object' and df[col].str.contains(',').any():
                    df[col] = df[col].str.replace(',', '.').astype(float)
                    print(f'- Заменил запятую на точку в столбце {col}')
                    print(f'--- Значения в столбце {col}: {df[col].unique()}\n')
                else:
                    print(f'- В столбце {col} замена не требуется')
        
        print('- Обработка завершена\n')
        return df

In [ ]:
class OutlierHandler(BaseEstimator, TransformerMixin):
    """Универсальный класс для обработки выбросов"""

    def __init__(
        self,
        skip_on_test=True,
        target_columns: Optional[List[str]] = None, 
        columns: Optional[List[str]] = None,
        method: str = 'IQR',
        action: str = 'winsorize',
        factor: float = 1.5,
        clip_quantiles: tuple = (0.01, 0.99),
        IQR_quantiles: tuple = (0.25, 0.75),
        extreme_factor: float = 3.0,
        min_valid_values: Optional[Dict[str, float]] = None,
        max_valid_values: Optional[Dict[str, float]] = None
    ):

        """
        Parameters:
        -----------
        columns: list of str optional, default=None (обрабатываются все числовые)
            Столбцы для обработки. Если None, обрабатываются все числовые столбцы.

        skip_on_test: bool, default=True
            Если True, то для тестовой выборки обрабатываются только target_columns (если они указаны), остальные колонки пропускаются. Если False, то обрабатываются все колонки
        
        target_columns: list of str optional, default=None 
            Список целевых колонок для тестовой выборки, которые нужно обработать. Если None, то на тесте ничего не обрабатывается.

        method: str, default='IQR'
            'IQR' или 'quantile'

        action: str, default='winsorize'
            'remove' (всю строку), 'nan', 'mean', 'clip' (замещение выбросов граничным значением), 'winsorize' (умная обработка)

        factor: float, default=1.5
            Множитель для IQR (только для IQR метода)    

        extreme_factor: float, default=3.0
            Множитель для экстремальных выбросов (только для winsorize)

        IQR_quantiles: tuple, default=(0.25, 0.75)
            Квантили для IQR метода

        clip_quantiles: tuple, default=(0.01, 0.99)
            Квантили для clip метода

        min_valid_values: dict, default=None
            Словарь с минимально допустимыми значениями для каждой колонки 

        max_valid_values: dict, default=None
            Словарь с максимально допустимыми значениями для каждой колонки
        """

        self.skip_on_test = skip_on_test
        self.target_columns = target_columns or []
        self.target_handling = bool(self.target_columns)  # флаг, указывающий, нужно ли обрабатывать только целевые колонки в тесте
        self.columns = columns
        self.method = method
        self.action = action
        self.factor = factor
        self.extreme_factor = extreme_factor
        self.clip_quantiles = clip_quantiles
        self.IQR_quantiles = IQR_quantiles
        self.bounds_dict = {}                   # словарь с границами, который был создан в методе fit() выбросов может и не быть
        self.means_dict = {}
        self.min_valid_values = min_valid_values
        self.max_valid_values = max_valid_values


    def fit(self, X, y=None, **kwargs):
        """Запоминает границы. Только для train выборки"""

        X_clean = X.copy()
    
        # Удаляем физически невозможные значения ПЕРЕД расчетом IQR
        if self.min_valid_values:
            for col, min_val in self.min_valid_values.items():
                if col in X_clean.columns:
                    X_clean = X_clean[X_clean[col] >= min_val]
        
        if self.max_valid_values:
            for col, max_val in self.max_valid_values.items():
                if col in X_clean.columns:
                    X_clean = X_clean[X_clean[col] <= max_val]

        cols = self.columns if self.columns else X.select_dtypes(include=[np.number]).columns
        
        if self.method == 'quantile':
            for col in cols:
                lower = X[col].quantile(self.clip_quantiles[0])
                upper = X[col].quantile(self.clip_quantiles[1])
                self.bounds_dict[col] = (lower, upper)
        else:  # IQR
            for col in cols:
                Q1 = X[col].quantile(self.IQR_quantiles[0])
                Q3 = X[col].quantile(self.IQR_quantiles[1])
                IQR = Q3 - Q1
                lower = Q1 - self.factor * IQR
                upper = Q3 + self.factor * IQR
                self.bounds_dict[col] = (lower, upper)
        
        if self.action == 'mean':
            for col in cols:
                self.means_dict[col] = X[col].mean()
        
        return self


    def transform(self, X, name=None):
        """Применяет обработку. Для test пропускает все, кроме target_columns"""

        X_transformed = X.copy()
        cols = self.columns if self.columns else X.select_dtypes(include=[np.number]).columns

        # Если test - обрабатываем только target_columns
        if self.skip_on_test and name and ('test' in name.lower() or 'val' in name.lower()) and self.target_handling:
            cols = [col for col in cols if col in self.target_columns]
            if not cols:
                print(f"- Пропускаю обработку выбросов для {name}")
                return X
            print(f"- Test выборка: обрабатываю только целевые колонки: {cols}")

        # БЛОК 1: Удаление физически невозможных значений (СНАЧАЛА)
        # Удаление физически невозможных значений (например, возраст < 0 или пробег > 1000000, пробег < 0, цена < 0, год  выпуск авто 1500 и т.д.)
        if self.min_valid_values:
            for col, min_val in self.min_valid_values.items():
                if col in cols and col in X_transformed.columns:
                    invalid_mask = X_transformed[col] < min_val
                    
                    if invalid_mask.any():
                        invalid_values = X_transformed.loc[invalid_mask, col]
                        count = invalid_mask.sum()
                        min_invalid = invalid_values.min()
                        max_invalid = invalid_values.max()
                        
                        print(f"\n⚠️ Обнаружены недопустимые значения в '{col}':")
                        print(f"   Количество: {count} строк")
                        print(f"   Недопустимый диапазон: [{min_invalid:.2f} - {max_invalid:.2f}]")
                        print(f"   Минимально допустимое значение: {min_val}")
                        print(f"   → Строки будут удалены как явный шум (независимо от IQR)")
                        
                        X_transformed = X_transformed[~invalid_mask]

        if self.max_valid_values:
            for col, max_val in self.max_valid_values.items():
                if col in cols and col in X_transformed.columns:
                    invalid_mask = X_transformed[col] > max_val
                    
                    if invalid_mask.any():
                        invalid_values = X_transformed.loc[invalid_mask, col]
                        count = invalid_mask.sum()
                        min_invalid = invalid_values.min()
                        max_invalid = invalid_values.max()
                        
                        print(f"\n⚠️ Обнаружены недопустимые значения в '{col}':")
                        print(f"   Количество: {count} строк")
                        print(f"   Недопустимый диапазон: [{min_invalid:.2f} - {max_invalid:.2f}]")
                        print(f"   Максимально допустимое значение: {max_val}")
                        print(f"   → Строки будут удалены как явный шум (независимо от IQR)")
                        
                        X_transformed = X_transformed[~invalid_mask]


        # Этот код не ищет выбросы — он просто проверяет, есть ли выбросы в каждой колонке.
        # Чтобы Вывести предупреждение ниже, в каких колонках найдены выбросы
        outlier_cols = []
        for col in cols:
            if col in self.bounds_dict:
                lower, upper = self.bounds_dict[col]
                outliers = (X_transformed[col] < lower) | (X_transformed[col] > upper)
                if outliers.any():
                    outlier_cols.append(col)
        
        if not outlier_cols:
            print('- Выбросы не обнаружены')
            return X_transformed
        
        print(f'\n- Обнаружены выбросы в столбцах: {outlier_cols}')
        print(f'- Метод: {self.method}, Действие: {self.action}\n')


        # ВЫВОД ГРАНИЦ ДЛЯ КАЖДОЙ КОЛОНКИ
        for col in outlier_cols:
            if col in self.bounds_dict:
                lower, upper = self.bounds_dict[col]
                print(f'  Нормальные пределы для {col}: [{lower:.2f} - {upper:.2f}]')
        print()
        

        # ВИНЗОРИЗАЦИЯ (умная обработка)
        if self.action == 'winsorize':
            print('- Применяю винзоризацию (мягкие → clip, экстремальные → remove)')
            X_transformed['outlier_status'] = 'normal'
            
            for col, (lower, upper) in self.bounds_dict.items():
                # Границы для экстремальных выбросов
                IQR = upper - lower
                extreme_lower = lower - self.extreme_factor * IQR
                extreme_upper = upper + self.extreme_factor * IQR
                
                # Мягкие выбросы → винзоризация (clipping)
                mild_outliers = ((X_transformed[col] < lower) & (X_transformed[col] >= extreme_lower)) | \
                               ((X_transformed[col] > upper) & (X_transformed[col] <= extreme_upper))
                X_transformed.loc[mild_outliers, 'outlier_status'] = 'mild'
                X_transformed.loc[mild_outliers, col] = X_transformed.loc[mild_outliers, col].clip(lower, upper)
                
                # Экстремальные выбросы → маркировка для удаления
                extreme_outliers = (X_transformed[col] < extreme_lower) | (X_transformed[col] > extreme_upper)
                X_transformed.loc[extreme_outliers, 'outlier_status'] = 'extreme'
            
            # Удаляем только экстремальные
            n_extreme = (X_transformed['outlier_status'] == 'extreme').sum()
            if n_extreme > 0:
                print(f'Удалено экстремальных выбросов: {n_extreme}')
                X_transformed = X_transformed[X_transformed['outlier_status'] != 'extreme']
            
            X_transformed = X_transformed.drop('outlier_status', axis=1)
        

        # Остальные действия (clip, nan, mean, remove)
        elif self.action == 'clip':
            for col, (lower, upper) in self.bounds_dict.items():
                X_transformed[col] = X_transformed[col].clip(lower, upper)
        
        elif self.action == 'nan':
            for col, (lower, upper) in self.bounds_dict.items():
                outliers = (X_transformed[col] < lower) | (X_transformed[col] > upper)
                X_transformed.loc[outliers, col] = np.nan
        
        elif self.action == 'mean':
            for col, (lower, upper) in self.bounds_dict.items():
                outliers = (X_transformed[col] < lower) | (X_transformed[col] > upper)
                X_transformed.loc[outliers, col] = self.means_dict[col]
        
        elif self.action == 'remove':
            if self.columns is None:
                mask = pd.Series([True] * len(X_transformed), index=X_transformed.index)
                for col, (lower, upper) in self.bounds_dict.items():
                    outliers = (X_transformed[col] < lower) | (X_transformed[col] > upper)
                    mask &= ~outliers
                X_transformed = X_transformed[mask]
                X_transformed = X_transformed.reset_index(drop=True)
            else:
                for col, (lower, upper) in self.bounds_dict.items():
                    outliers = (X_transformed[col] < lower) | (X_transformed[col] > upper)
                    X_transformed = X_transformed[~outliers]
                    X_transformed = X_transformed.reset_index(drop=True)
        
        return X_transformed


    def fit_transform(self, X, y=None, **fit_params):   # name=None передается через **fit_params, чтобы не ломать сигнатуру метода
        return self.fit(X, y).transform(X, **fit_params)


In [ ]:
class ImplicitDuplicatesViewer(BaseEstimator, TransformerMixin):
    """Выводит уникальные значения каждого столбца для визуального определения неявных дубликатов только для тренировочной выборки"""

    def __init__(self, skip_on_test=True, columns: List[str] = None):
        """Инициализация определеителя неявных дуликатов"""
        self.columns = columns
        self.skip_on_test = skip_on_test


    def fit(self, X: pd.DataFrame, y: None = None):
        return self


    def transform(self, X: pd.DataFrame, y: None = None, name=None):
        """Выводит уникальные значения каждого столбца для визуального определения неявных дуликатов"""

        if self.skip_on_test and name and ('test' in name.lower() or 'val' in name.lower()):
            print(f"- Пропускаю проверку на неявные дубликаты для {name}")
            return X


        print('- Выполняю поиск неявных дубликатов в нечисловых столбцах')

        df = X.copy()

        if self.columns:
            for col in self.columns:
                if col in df.columns:
                    print(f'- Уникальные значения в столбце {col}: {sorted(X[col].unique().tolist())}\n')
            return X
        else: 
            # columns = X.select_dtypes(exclude=[np.number]).columns # проверить только нечисловвые ячейки
            for col in df.columns:
                print(f'- Уникальные значения в столбце {col}: {sorted(X[col].unique().tolist())}\n')
            return X
        
    def fit_transform(self, X, y: None = None, **fit_params):   # name=None передается через **fit_params, чтобы не ломать сигнатуру метода
        return self.fit(X, y).transform(X, **fit_params)

In [ ]:
class DuplicateRemover(BaseEstimator, TransformerMixin):
    """Класс для удаления дубликатов только для тренировочной выборки"""

    def __init__(self, skip_on_test=True, columns: List[str] = None):
        """Инициализация удалителя дубликатов"""
        self.columns = columns
        self.skip_on_test = skip_on_test


    def fit(self, X: pd.DataFrame, y: None = None):
        return self


    def transform(self, X: pd.DataFrame, y: None = None, name=None):
        """Удаляет дубликаты только в тренировочной выборке"""

        if self.skip_on_test and name and ('test' in name.lower() or 'val' in name.lower()):
            print(f"- Пропускаю удаление дубликатов для {name}")
            return X
        
        duplicate_count = X.duplicated().sum()

        if duplicate_count:
            print(f'- Выявлено {duplicate_count} дубликатов')
            print('- Выполняю удаление дубликатов\n')

            if self.columns:
                X = X.drop_duplicates(subset=self.columns)                
            else:
                X = X.drop_duplicates()
            
            remaining_duplicates = X.duplicated().sum()
            print(f'- Осталось {remaining_duplicates} дубликатов\n')
        else:
            print('- Дубликатов не выявлено\n')
        
        return X  # ← теперь возвращаешь очищенный X

    
    def fit_transform(self, X, y: None = None, **fit_params):   # name=None передается через **fit_params, чтобы не ломать сигнатуру метода
        return self.fit(X, y).transform(X, **fit_params)

In [ ]:
class MissingValueHandler(BaseEstimator, TransformerMixin):
    """Класс для обработки пропущенных значений в данных. Возможные варианты параметра strategy: mean, median, mode, drop, unknown. По умолчанию drop. Для обеих выборок."""

    def __init__(self, skip_on_test=False, strategy='mean', fill_value=None):
        """Инициализация обработчика пропущенных значений. По умолчанию заполняет средним значением.
        """
        self.skip_on_test = skip_on_test
        self.strategy = strategy
        self.fill_value = fill_value
        self.fill_values_ = {}  # для хранения значений для заполнения

    def fit(self, X: pd.DataFrame, y: None=None):
        """Запоминает значения из train"""
        if self.strategy == 'mean':
            self.fill_values_ = X.select_dtypes(include=[np.number]).mean().to_dict()
        elif self.strategy == 'median':
            self.fill_values_ = X.select_dtypes(include=[np.number]).median().to_dict()
        elif self.strategy == 'mode':
            self.fill_values_ = {col: X[col].mode()[0] for col in X.columns}
        elif self.strategy == 'unknown':
            self.fill_values_ = {}  # для unknown не нужно запоминать
        elif self.strategy == 'drop':
            self.fill_values_ = {}  # для drop не нужно запоминать
    
        return self
        
    def transform(self, X: pd.DataFrame, y: None = None, name=None, **fit_params):
        """Применяет значения, которые запомнил из train. Заполняет пропущенные значения или удаляет строки в тестовой выборке. режимы: mean, median, mode, drop"""
        
        df = X.copy()

        if self.strategy == 'unknown':
            for col in df.select_dtypes(include='object').columns:
                if df[col].isnull().any():
                    df[col] = df[col].fillna('unknown')

        # Стратегия drop — удаление строк
        if self.strategy == 'drop':
            null_count = df.isna().sum().sum()
            if null_count > 0 or df.eq(" ").any().any():
                print('- Нашел пропуски в данных\n')
                null_string_count = len(df[df.isna().any(axis=1)])
                display(df[df.isna().any(axis=1)])
                if len(df[df.eq(" ").any(axis=1)]) != 0:
                    display(df[df.eq(" ").any(axis=1)])
                
                null_string_percentage = null_string_count / len(df) * 100
                if null_string_percentage < 10:
                    print(f'- Выявлено {null_count} пропусков в {null_string_count} строках ({null_string_percentage:.2f}%). Удаляю\n')
                    df = df.dropna()
                    df = df[~df.eq(" ").any(axis=1)]
                    print(f'- Осталось {df.isna().sum().sum()} пропусков\n')
            else:
                print('- Пропусков не найдено\n')
            return df
        
        # Остальные стратегии — заполнение запомненными значениями
        for col, fill_value in self.fill_values_.items():
            if col in df.columns and df[col].isnull().any():
                count = df[col].isnull().sum()
                percent = (count / len(df)) * 100
                print(f"- '{col}': {count} пропусков ({percent:.1f}%). Заполняю '{self.strategy}' → {fill_value}")
                df[col] = df[col].fillna(fill_value)
        
        return df

    def fit_transform(self, X: pd.DataFrame, y: None = None, **fit_params):  # name=None передается через **fit_params, чтобы не ломать сигнатуру метода
        return self.fit(X, y).transform(X, **fit_params)
        

In [ ]:
class ColumnRemover(BaseEstimator, TransformerMixin):
    """Удаляет лишние колонки, переданные в списке. Работает как для train, так и для test выборки""" 

    def __init__(self, columns: List[str], skip_on_test=False):
        self.columns = columns
        self.skip_on_test=skip_on_test

    def fit(self, X: pd.DataFrame, y: None=None, name=None):
        # Просто сохраняем информацию о столбцах для удаления
        return self

    def transform(self, X: pd.DataFrame, name=None):

        df = X.copy()

        for col in self.columns:
            if col in df.columns:
                df = df.drop(col, axis=1)
                print(f'- Удалил столбец {col}')
        return df

    def fit_transform(self, X: pd.DataFrame, y: None=None, **fit_params):  # name=None передается через **fit_params, чтобы не ломать сигнатуру метода
        return self.fit(X, y).transform(X, **fit_params)

In [ ]:
class FloatToIntChanger(BaseEstimator, TransformerMixin):
    """
    Преобразует дробные значения в целочисленные (режим Multiplie - по умолчанию: 
    умножает на 100 и сохраняет как Int, simple: без умножения меняет тип) на основе переданного списка столбцов
    """    

    def __init__(self, columns, strategy, skip_on_test=False):
        self.columns = columns
        self.strategy = strategy
        self.skip_on_test = False


    def fit(self, X: pd.DataFrame, y=None):
        # Вызывается только для train. Просто сохраняем информацию о столбцах формально
        if isinstance(X, pd.DataFrame):
            self.feature_names_in_ = X.columns.tolist()
        else:
            # Если на входе массив, генерируем имена колонок
            self.feature_names_in_ = [f"col_{i}" for i in range(X.shape[1])]
        return self
    

    def transform(self, X: pd.DataFrame, y=None, name=None):

        if isinstance(X, pd.DataFrame):
            df = X.copy()
        else:
            # Восстанавливаем DataFrame из массива
            df = pd.DataFrame(X, columns=self.feature_names_in_)

        if self.strategy == 'simple':
            for col in self.columns:
                if col in df.columns:
                    print(f'\n - Меняю тип на int столбце {col}')
                    df = df[col].astype('int')
            return df

        if self.strategy == 'multiplie':
            for col in self.columns:
                if col in df.columns:
                    print(f'\n - Значения в столбце {col} умножаю на 100 ')
                    df[col] = (df[col] * 100).astype('int')
                else:
                    print(f'- Колонка {col} не найдена в данных')
            return df

    def fit_transform(self, X: pd.DataFrame, y=None, **fit_params):
        return self.fit(X, y).transform(X, **fit_params)

In [ ]:
class EDAPreprocessor:
    """
    Основной класс пайплайна для предобработки данных
    """

    def __init__(self, func: Callable[..., Any] | None = None):
        self.steps = []  # список шагов предобработки, которые будут выполняться в пайплайне
        self.fitted_transformers = {}  # для хранения обученных трансформеров (запоминает что-то и делает)
        
    def add_mistake_corrector(
                                self, 
                                columns: List[str] | None = None,
                                values_dict: dict | None = None, 
                                func: Callable[..., Any] | None = None, 
                                strategy: str | None = None, 
                                skip_on_test: bool = False,
                                step_name: str =' Преобразование некорректных данных'):
        """Добавляет шаг исправления ошибок в препроцессор, принимает на вход список колонок, в которых произвести замены, словарь с неверными и верными значениями"""
        
        mistake_corrector = MistakeCorrector(
                                                columns=columns, 
                                                values_dict=values_dict, 
                                                func=func,
                                                strategy=strategy
            )
        self.steps.append((step_name, mistake_corrector))
        return self

    def add_column_remover(
            self, 
            columns: List[str], 
            skip_on_test: bool = False,
            step_name: str = 'Удаление столбцов'):
        
        column_remover=ColumnRemover(columns=columns)
        self.steps.append((step_name, column_remover))
        return self

    def add_float_to_int_changer(
            self, 
            columns:List[str] | None = None,
            strategy='multiplie', 
            skip_on_test: bool = False,
            step_name='Преобразование дробных чисел в целочисленное'):
        float_to_int_changer=FloatToIntChanger(columns, strategy)
        self.steps.append((step_name, float_to_int_changer))
        return self

    def add_decimal_point_changer(
            self, 
            columns:List[str] | None = None, 
            skip_on_test: bool = False,
            step_name='Замена запятой на точку в дробных числах при необходимости'):
        decimal_point_changer = DecimalPointChanger(columns)
        self.steps.append((step_name, decimal_point_changer))
        return self


    def add_missing_value_handler(
            self, 
            strategy='drop', 
            fill_value=None, 
            skip_on_test: bool = False,
            step_name='Проверка пропущенных значений'):
        """Добавляет обработчик пропущенных значений в препроцессор"""
        missing_handler = MissingValueHandler(strategy=strategy, fill_value=fill_value)
        self.steps.append((step_name, missing_handler))
        return self    


    def add_outlier_handler(
            self, 
            columns: Optional[List[str]] = None,
            target_columns:Optional[List[str]] = None,
            method: str = 'IQR',
            action: str = 'winsorize',
            factor: float = 1.5,
            extreme_factor: float = 3.0,
            min_valid_values: Optional[Dict[str, float]] = None,
            max_valid_values: Optional[Dict[str, float]] = None,
            skip_on_test: bool = True,
            step_name: str = 'Проверка на наличие выбросов'):
        '''Добавляет шаг обработки выбросов'''
        outlier_handler = OutlierHandler(columns=columns, method=method, action=action, factor=factor, extreme_factor=extreme_factor, min_valid_values=min_valid_values, max_valid_values=max_valid_values, skip_on_test=skip_on_test)
        self.steps.append((step_name, outlier_handler))
        return self

 
    
    def add_drop_duplicates(
            self, 
            skip_on_test: bool = True,
            step_name='Проверка на наличие явных дубликатов'):
        """Добавляет шаг удаления дуликатов"""
        duplicate_remover = DuplicateRemover(skip_on_test=skip_on_test)
        self.steps.append((step_name, duplicate_remover))
        return self
    

    def add_implicit_duplicates_viewer(
            self, 
            skip_on_test: bool = True,
            columns:List[str] | None = None, 
            step_name='Отображение неявных дубликатов и проверка на неоднородность данных'):
        """Добавляет шаг отображения уникальных значений каждого столбца для выявления неявных дуликатов визуально."""
        implict_duplicates_viewer = ImplicitDuplicatesViewer(columns=columns, skip_on_test=skip_on_test)
        self.steps.append((step_name, implict_duplicates_viewer))
        return self
    

    def add_custom_transformer(self, step_name: str, transformer):
        """Добавляет пользовательский трансформер в пайплан предобраотки"""
        self.steps.append((step_name, transformer))
        return self
    

    def fit_transform(self, X: pd.DataFrame, name, y: None=None):
        """Обучает все трансформеры пайплайна (запоминает единые параметры и условия исполнения - консистентность). Принимает df и целевую переменную опционально."""

        df = X.copy()

        for i, (step_name, transformer) in enumerate(self.steps):
            print(f'\nИсполнение шага {i+1}: {step_name}')
            df = transformer.fit_transform(df, name=name)
            self.fitted_transformers[step_name] = transformer

        return df

In [ ]:
EDA_Preprocessor_pipeline = (
    EDAPreprocessor()  
    .add_mistake_corrector(columns=['registration_month'], values_dict={0: 'median'}, strategy='auto', skip_on_test=False)
    .add_mistake_corrector(columns=['fuel_type'], values_dict={'gasoline': 'petrol'}, strategy='auto', skip_on_test=False)
    .add_column_remover(columns=['date_crawled', 'date_created', 'last_seen', 'number_of_pictures'], skip_on_test=False)
    .add_decimal_point_changer()                            
    .add_outlier_handler(skip_on_test=True, target_columns=['price'], method='IQR', action='winsorize', factor=1.5, extreme_factor=3.0, min_valid_values={'price': 1, 'power': 1, 'registration_year': 1920}, max_valid_values={'power': 1900, 'registration_year': 2016}) 
                                           # IQR, clip   # remove, nan, mean, clip, winsorize                                  
    .add_missing_value_handler(skip_on_test=False, strategy='unknown')   # на тестовой только заполнять, никогда не удалять         
                                                   # mean, median, mode, unknown, drop
    .add_drop_duplicates(skip_on_test=True)                                    
    .add_implicit_duplicates_viewer(columns=None, skip_on_test=True)           

)

print('Вот таким у нас получился предобработчик данных.\n')

print("Шаги в пайплайне:\n")
for i, (name, step) in enumerate(EDA_Preprocessor_pipeline.steps):
    print(f"{i+1}. {name}: {step}\n")

In [ ]:
def run_preprocessor(df_dict=df_dict):
    """Производит предобработку всех датафреймов в цикле в пайплайне"""
    for name, df in df_dict.items():
        print('=' * 50)
        print(f' =>  Обработка датафрейма {name}')
        print('=' * 50)
        df_dict[name] = EDA_Preprocessor_pipeline.fit_transform(df, name)
        globals()[name] = df_dict[name]  # Перезаписывает глобальную переменную
        print(f'\nПроверка датафрейма {name}')
        display(df_dict[name].head())
        display(df_dict[name].info())
        print(f'Обработка датафрейма {name} завершена.\n\n')

    # Объединить три выборки
    df = pd.concat([train_df, test_df, val_df], ignore_index=True)

    print('Выведем описательную статистику по всем трем датиафреймам после обработки в сумме, чтобы посмотреть, что осталось в итоге.')
    print('ОЦЕНКА СТАТИСТИЧЕСКИ ОПИСАТЕЛЬНОЙ СТАТИСТИКИ ПО ВСЕМ ДАТАФРЕЙМАМ В СУММЕ ПОСЛЕ ПРЕДОБРАБОТКИ\n')

    # Общий describe
    display(df.describe())    

run_preprocessor(df_dict)

### **3.5. Классификация признаков по типам**

Перед построением распределений признаков и обучений модели необходимо классифицировать их по способу обработки, а не по математической классификации: 

- **числовые**, 
- **категориальные**

**1. Числовые** — используются как есть:

- `RegistrationYear` — непрерывный

- `Power` — непрерывный

- `Kilometer` — дискретный (но обрабатывается как непрерывный)

- `RegistrationMonth` — дискретный

- `PostalCode` — числовой (но может быть категориальным, регион влияет на цену, но 99к уникальных значений, можно агрегировать)

**2. Категориальные** — требуют кодирования:

- `VehicleType` — номинальный (sedan, suv, coupe...)

- `Gearbox` — номинальный (manual, auto)

- `Model` — номинальный (golf, passat...)

- `FuelType` — номинальный (petrol, diesel...)

- `Brand` — номинальный (volkswagen, audi...)

- `Repaired` — бинарный (yes/no)

При визуализации признаков, их следует  делить на две большие группы `Дискретные` и `Непрерывные`.

**1) Дискретные включают:**

- бинарные;
- категориальные 
- целые числа (месяц, год, количество, поддающееся несложному подсчету, когда можно перечислить все варианты)

дискретные в нашем случае: `RegistrationYear`, `RegistrationMonth`, `Kilometer` (binned), `VehicleType`, `Gearbox`, `Brand`, `NotRepaired`. Обращаем внимание, что пробег авто прдствален в бинированном виде в признаке Kilometer и имеет всего пять значений [87500, 90000, 100000, 125000, 150000].

Для визуализщации дискретных признаков рекомендуется использовать countplot из seaborn.

**2) Непрерывные включают:**

- дробные числа;
- измерения;
- время
- когда бесконечно много значений в диапазоне.

Непрерывные признаки в нашем случае: `Price`, `Power`, `PostalCode` (218k уникальных значений)

Призанки делятся по-разному в зависмости от решаемой задачи на данный момент:

- при визуализации признаки принято делить на длискретные и непрерывные.

- при обучении модели делить на числовые и категориальные.

## **3.6. Оценка распределения признаков**

### **3.6.1. Распределение дискретных признаков**

In [ ]:
# Визуализация дискретных признаков
discrete_features = ['registration_year', 'registration_month', 'kilometer', 
                     'vehicle_type', 'gearbox', 'brand', 'repaired', 'fuel_type']

fig, axes = plt.subplots(4, 2, figsize=(20, 16))
axes = axes.flatten()

for idx, feature in enumerate(discrete_features):
    sns.countplot(data=train_df, x=feature, ax=axes[idx]) # , order=train_df[feature].value_counts().index
    axes[idx].set_title(f'Распределение: {feature}')
    axes[idx].tick_params(axis='x', rotation=70)
    
# axes[-1].axis('off')  # Скрыть последний пустой subplot
plt.tight_layout()
plt.show()


In [ ]:
#### **3.6.1.1. Распределение категориальных признаков**

Категориальные признаки входят в состав дискретных признаков.

#### **3.6.1.2. Распределение дискретного высококардинального признака с 218k значений через countplot из seaborn.**

In [ ]:
train_df['region'] = train_df['postal_code'] // 1000
test_df['region'] = test_df['postal_code'] // 1000
val_df['region'] = val_df['postal_code'] // 1000

features = ['region', 'model']

fig, axes = plt.subplots(2, 1, figsize=(25, 6))

for idx, feature in enumerate(features):
    sns.countplot(data=train_df, x=feature, ax=axes[idx], order=train_df[feature].value_counts().index)
    axes[idx].set_title(f'Распределение по признаку {feature}')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Количество')
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

#### **3.6.1.3. Облако слов для категориальных признаков**

Построим облако слов для категориальных признаков.

In [ ]:
# Объединяем строки в единый текст для каждого столбца
brands_text = ' '.join(df['brand'].astype(str))
models_text = ' '.join(df['model'].astype(str))

# Создаем облако слов для брендов
wc_brands = WordCloud(width=4000, height=3000).generate(brands_text)

# Создаем облако слов для моделей автомобилей
wc_models = WordCloud(width=4000, height=3000).generate(models_text)

# Отображаем оба облака слов
plt.figure(figsize=(12, 6))

# Облако слов брендов
plt.subplot(1, 2, 1)
plt.imshow(wc_brands, interpolation='bilinear')
plt.title('Облако брендов')
plt.axis("off")

# Облако слов моделей
plt.subplot(1, 2, 2)
plt.imshow(wc_models, interpolation='bilinear')
plt.title('Облако моделей')
plt.axis("off")

plt.show()

In [ ]:
### **3.6.2. Распределение непрерывных признаков**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(data=train_df, x='price', bins=50, kde=True, ax=axes[0])
axes[0].set_title('Распределение цены')

sns.histplot(data=train_df, x='power', bins=50, kde=True, ax=axes[1])
axes[1].set_title('Распределение мощности')

plt.tight_layout()
plt.show()


## **3.7. Расшифровка визуализации распределения признаков**

Несмотря на предобработку данных и удаление выбросов в частности, 

практически все признаки имеют скошенное распределение: Бренд, год регистрации, Тип топлива, тип авто, мощномть, пробег.

Нормальное распределение у года регистрации. 

Вопросы остались по мощности авто что считать выбросами, будет ли данная модель предсказывать стоимость спорткаров или нет? Выбросы по мощности свыше 1500 л.с. мы удалили.

К числовым признакам, имеющим скошенное распределение мы применим RobustScaler

К категориальным признакам мы применим категорирование OneHotEncoder

Месяц регистрации особой ценности признак не несет.


## **3.8. Оценка статистических предобработанных данных** 

In [ ]:
df.describe()

расшифровка дискрайба

**Рекомендации для обучения модели**

In [ ]:
train_df.head()

## **3.9. Корреляция данных**

In [ ]:
interval_cols = interval_cols = ['price', 'power', 'kilometer', 'postal_code', 'age'] # только дробные числа, или которые не в силах посчитать руками (если очень много значений, значит интервальный тип)

phik_corr = train_df.phik_matrix(interval_cols=interval_cols)
phik_corr     

In [ ]:
plt.figure(figsize=(15, 15))
sns.heatmap(phik_corr.round(2), annot=True, cmap='coolwarm')
plt.show()

**Мультиколлинеарность наблюдается среди признаков:**

- `RegistrationYear` и `Age` (возраст = 2016 - год) - почти полная корреляция
- `Brand` и `Model` 
- `Kilometer` & `Kilometer_Max`

**Высокая корреляция с целевым признаком у:**

- `Age` - 0.67
- `RegistrationYear` - 0.63  (но модель lgbm с ним обучается хуже)
- `Model` - 0.58
- `Power` - 0.55

In [ ]:
sns.pairplot(train_df[interval_cols], plot_kws={'alpha': 0.3})

## **3.10. Проверка данных на неоднородность: выявление нелинейных связей**

Мы надеемся, что выявление скрытых связей поможет повысить качество обучения.

**Мы применили:**

1. **Тесты Левена и Бартлетта** для проверки однородности дисперсий; 
2. **Simpson's Paradox:** когда общая корреляция и внутри груцппы имеют противопаоложные знаки;
3. Группировочный анализ;
4. Стабильность корреляций;
5. Скрытые кластеры;
6. Нелинейные связи;
7. Взаимодействия.

**Выводы:**
Скрытых взаимосвзей не обнаружено.  Казалось бы, что-то удалось найти, но при обучении модели они дают хуже результат, поэтому от этой информации пришлось отказаться при обучении модели. Блок удален за ненадобностью.

## **4. Feature Enginering**

## **4.1. Новые признаки на основе анализа неоднородности**

Создание кластера.

Благоджаря кластеризации, модель понимает контекст - одинаковые характеристики могут означать разную цену в зависимости от сегмента.

Кластеризация должна быть только на train, затем применяется к test через predict().

In [ ]:
# Для кластеризации используем ВСЕ важные признаки
cluster_features = ['power', 'kilometer', 'age', 'price']  # можно добавить Region, но там много уникальных значений, может не сработать
X_cluster = train_df[cluster_features].fillna(train_df[cluster_features].median())

# Обучение
kmeans = KMeans(n_clusters=10, random_state=42)
train_df['cluster'] = kmeans.fit_predict(X_cluster)
X_test_cluster = test_df[cluster_features].fillna(train_df[cluster_features].median())
test_df['cluster'] = kmeans.predict(X_test_cluster)


Проведем ради интереса анлиз кластеров

In [ ]:
# Анализ кластеров
cluster_analysis = train_df.groupby('cluster').agg({
                                    'price': ['mean', 'median', 'std', 'count'],
                                    'power': 'mean',
                                    'age': 'mean',
                                    'kilometer': 'mean',
                                    'brand': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'mixed'
}).round(0)

cluster_analysis.columns = ['Price_mean', 'Price_median', 'Price_std', 'Count', 'Power_avg', 'Age_avg', 'Km_avg', 'Top_Brand']
print(cluster_analysis.sort_values('Price_mean', ascending=False))

# Сравним разброс
print(f"Общий std цены: {train_df['price'].std():.0f}€")
print(f"Средний std внутри кластеров: {cluster_analysis['Price_std'].mean():.0f}€")

Если средний std внутри кластеров < общего std - значит, кластеризация работает.

подправить код убрать лишнее

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Предположим, что train_df - ваш DataFrame с исходными данными
# Выбираем нужные признаки для кластеризации
cluster_features = ['power', 'kilometer', 'age', 'price']
X_cluster = train_df[cluster_features].fillna(train_df[cluster_features].median())  # Заполняем пропуски медианой

# Нормализация признаков перед кластеризацией (K-means чувствителен к масштабированию данных)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Кластеризация методом K-means
n_clusters = 10  # Число кластеров
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Расчет среднего коэффициента Силуэта
silhouette_avg = silhouette_score(X_scaled, labels)
print(f'Средний коэффициент Силуэта: {silhouette_avg:.3f}')

анализ кластеризации работает ли

## **4.2. Стандартный Feature Engineering**

Всевозможные сложения и перемножения и деления  признаков качество модели lgbm не улучшили.

# **5. Подготовка и обучение модели модели**

### **5.1. Отбор признаков с правильной корреляцией**

Удалим признаки с мультиколинеарностью и низкой корреляцией.

In [ ]:
# 1. Отбор по корреляции с таргетом
target_corr = phik_corr['price'].abs()
selected = target_corr[(target_corr >= 0.01) & (target_corr <= 0.9)].index.tolist()

if 'price' in selected:
    selected.remove('price')

# 2. Сортируем по убыванию корреляции с таргетом (приоритет важным признакам)
selected_sorted = sorted(selected, key=lambda x: target_corr[x], reverse=True)

# 3. Удаляем мультиколлинеарность, сохраняя более важные
corr_matrix = phik_corr.loc[selected, selected]
to_drop = set()

for i, col1 in enumerate(selected_sorted):      # проходит по признакам, отсортированным по важности (корреляции с таргетом)
    if col1 in to_drop:                         # пропускает уже помеченные на удаление
        continue
    for col2 in selected_sorted[i+1:]:          # сравнивает каждый признак только с последующими (избегает дублирования) (индекс предыдущего +1)
        if col2 in to_drop:                     # пропускает уже помеченные на удаление
            continue
        if corr_matrix.loc[col1, col2] > 0.9:   # если корреляция между признаками > 0.9
            to_drop.add(col2)                   # # Удаляем col2 (у него корреляция с таргетом слабее)         

selected_features = [f for f in selected_sorted if f not in to_drop]

print(f"Отобрано: {len(selected_features)}")
print(f"Удалено: {len(to_drop)}")
print(f"\nТоп признаков по корреляции с таргетом:")
for f in selected_features:
    print(f"  {f:<20}: {target_corr[f]:.5f}")

# Классификация типов
feature_types = {
    'binary': [c for c in selected_features if train_df[c].nunique() == 2],
    'ordinal': [c for c in selected_features if 3 <= train_df[c].nunique() <= 5 and train_df[c].dtype in ['int64', 'float64'] and train_df[c].apply(lambda x: x == int(x) if pd.notna(x) and isinstance(x, (int, float)) else True).all()],
    'continuous': [c for c in selected_features if train_df[c].nunique() > 5],
    'nominal': [c for c in selected_features if train_df[c].dtype == 'object' or (train_df[c].dtype in ['int64', 'float64'] and train_df[c].nunique() <= 5)]

}

for ftype, cols in feature_types.items():
    print(f"{ftype.capitalize()}: {cols}")

## **5.2. Классификация и стандартизация признаков**

In [ ]:
#  классификация типов из отобранных признаков
def create_preprocessor():
    return ColumnTransformer([
        ('std', RobustScaler(), [
            'age', 
            'power', 
            'kilometer',
            # 'region' 
            
            ]), 
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), [
            'model', 
            'vehicle_type', 
            'gearbox', # с ним lr лучше, lgbm хуже, без него lgbm лучше
            'fuel_type', 
            'repaired', 
            'brand',
            # 'cluster'
            ])
    ], remainder='drop')


**Для обучения модели рассмотрим две модели:**

- **`Ridge`** - вместо более простого базового LinearRegression

**Преимущества `Ridge`:**

✅ Регуляризация — борется с переобучением

✅ Устойчивость к мультиколлинеарности — когда признаки коррелируют

✅ Стабильность — меньше чувствителен к выбросам

✅ Лучше обобщает — особенно при большом количестве признаков


- **`LightGBM`** - для примера градиентного бустинга.

✅ В 10-20 раз быстрее XGBoost

✅ Часто лучше CatBoost

✅ Использует меньше RAM

✅ Внутренний свой SelectKBest

✅ Работает с категориями БЕЗ One-Hot Encoding

## **5.3. Обучение модели**

In [ ]:
# Подготовка данных
X_train = train_df.drop('price', axis=1)
y_train = train_df['price']

X_test = test_df.drop('price', axis=1)
y_test = test_df['price']

X_val = val_df.drop('price', axis=1)
y_val = val_df['price']

# 1. Ridge регрессия 
pipeline_ridge = Pipeline([
                            ('preprocessor', create_preprocessor()),
                            ('model', Ridge(alpha=10, random_state=42))  # alpha сам отбирает важные признаки через кэфициенты
])

param_grid_ridge = {
                    'model__alpha': [0.1, 1, 10, 100, 1000],  # Сила регуляризации
                    'model__fit_intercept': [True, False],
                    'model__solver': ['auto', 'svd', 'cholesky', 'lsqr']
}

grid_ridge_model = GridSearchCV(
                                pipeline_ridge, 
                                param_grid_ridge, 
                                cv=KFold(n_splits=5, shuffle=True, random_state=42),
                                scoring='neg_root_mean_squared_error', 
                                n_jobs=-1
)

grid_ridge_model.fit(X_train, y_train)
y_pred_ridge = grid_ridge_model.predict(X_val) # X_test заменили на X_val для оценки на валидационной выборке, так как тестовая используется только для финальной проверки модели и не должна влиять на выбор гиперпараметров.


# Извлечение времени
best_idx = grid_ridge_model.best_index_
fit_time = grid_ridge_model.cv_results_['mean_fit_time'][best_idx]
predict_time = grid_ridge_model.cv_results_['mean_score_time'][best_idx]

rmse_ridge = np.sqrt(mean_squared_error(y_val, y_pred_ridge))  # y_test заменили на y_val для оценки на валидационной выборке, так как тестовая используется только для финальной проверки модели и не должна влиять на выбор гиперпараметров.

print(f"Ridge | RMSE: {rmse_ridge:.2f} € | Fit: {fit_time:.2f}s | Predict: {predict_time:.2f}s | Refit: {grid_ridge_model.refit_time_:.2f}s | Best params: {grid_ridge_model.best_params_}")

# 2. LightGBM 
pipe_lgb = Pipeline([
                    ('preprocessor', create_preprocessor()),
                    ('feature_selection', SelectFromModel(lgb.LGBMRegressor(n_estimators=50), threshold='median')),
                    ('model', lgb.LGBMRegressor(random_state=42, verbose=-1)),

])

param_grid_lgb = {
                'model__n_estimators': [200, 220, 230, 240, 250],
                'model__max_depth': [10, 13],
                'model__learning_rate': [0.1, 0.11],
                'model__verbose': [-1],
            }


grid_lgbm_model = GridSearchCV(
                                pipe_lgb, 
                                param_grid_lgb, 
                                cv=KFold(n_splits=5, shuffle=True, random_state=42),
                                scoring='neg_root_mean_squared_error', 
                                n_jobs=-1,
                                refit=True
    )

grid_lgbm_model.fit(X_train, y_train)
y_pred_lgbm = grid_lgbm_model.predict(X_val)  # X_test заменили на X_val для оценки на валидационной выборке, так как тестовая используется только для финальной проверки модели 

# Извлечение времени
best_idx = grid_lgbm_model.best_index_
fit_time = grid_lgbm_model.cv_results_['mean_fit_time'][best_idx]
predict_time = grid_lgbm_model.cv_results_['mean_score_time'][best_idx]


rmse_lgbm = np.sqrt(mean_squared_error(y_val, y_pred_lgbm)) # y_test заменили на y_val для оценки на валидационной выборке, так как тестовая используется только для финальной проверки модели

print(f"LightGBM | RMSE: {rmse_lgbm:.2f} € | Fit: {fit_time:.2f}s | Predict: {predict_time:.2f}s | Refit: {grid_lgbm_model.refit_time_:.2f}s | Best params: {grid_lgbm_model.best_params_}")



Сохраним лучшую модель в переменную `best_model`

In [ ]:
best_model = grid_lgbm_model.best_estimator_.named_steps['model'] if rmse_lgbm < rmse_ridge else grid_ridge_model.best_estimator_.named_steps['model']

## **5.4. Извлечение лучших признаков из модели**

Попробуем улучшить полученные результаты.

Извлечем наиболее влиятельные признаки и попробуем переобучить модель только на них, если результат окажется хуже - откажемся от переобучяенной модели.

Извлечем Feature Importance из лучшей модели чтобы сделать ее еще лучше.

In [ ]:
# 2. Извлечение feature importance для лучшей модели (только для LightGBM, так как Ridge не предоставляет встроенный способ извлечения важности признаков, а его коэффициенты сложно интерпретировать из-за регуляризации)
feature_names = grid_lgbm_model.best_estimator_.named_steps['preprocessor'].get_feature_names_out()

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-30 важных признаков:")
print(importance_df.head(30))



In [ ]:
# Визуализация топ-30
plt.figure(figsize=(10, 8))
plt.barh(importance_df.head(30)['feature'], importance_df.head(30)['importance'])
plt.xlabel('Importance')
plt.title('Топ-30 важных признаков')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

Три наиболее важных признака:
- возраст
- мощность 
- пробег

**Признаки с наименьшим влиянием на цену:**

- частные модели каких-то брендов 

In [ ]:
# Визуализация bottom-30
plt.figure(figsize=(10, 8))
plt.barh(importance_df.tail(30)['feature'], importance_df.tail(30)['importance'])
plt.xlabel('Importance')
plt.title('Bottom-30 неважных признаков')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# 3. Выбор топ-N признаков (например, топ-30)
top_n = 30
top_features = importance_df.head(top_n)['feature'].tolist()
top_indices = [list(feature_names).index(f) for f in top_features]

print(f"\nОтобрано {top_n} признаков")

# 4. Трансформация данных с отбором признаков
X_train_transformed = grid_lgbm_model.best_estimator_.named_steps['preprocessor'].transform(X_train)
X_test_transformed = grid_lgbm_model.best_estimator_.named_steps['preprocessor'].transform(X_test)

X_train_selected = X_train_transformed[:, top_indices]
X_test_selected = X_test_transformed[:, top_indices]

# 5. Переобучение модели на отобранных признаках
# Получить лучшую модель
best_model = grid_lgbm_model.best_estimator_.named_steps['model']

# Создать копию с теми же параметрами
model_refit_temp = lgb.LGBMRegressor(**best_model.get_params())
model_refit_temp.fit(X_train_selected, y_train)
y_pred_selected = model_refit_temp.predict(X_test_selected)

rmse_selected = np.sqrt(mean_squared_error(y_test, y_pred_selected))

print(f"\nМодель с {top_n} признаками RMSE: {rmse_selected:.2f}")
print(f"Разница: {rmse_selected - rmse_lgbm:+.2f}")


Модель с отобранным топом признаков оказалась хуже, поэтому не будем отбирать топ признаков.

## **5.5. Дообучение модели на извлеченных признаках**

RMSE Показывает среднюю ошибку в евро

In [ ]:
# Проверим среднюю цену
print(f"Средняя цена: {y_train.mean():.2f}€")
print(f"RMSE LightGBM: {rmse_lgbm:.2f}€ ({rmse_lgbm/y_train.mean()*100:.1f}% от средней)")

In [ ]:
from sklearn.dummy import DummyRegressor

# Предсказания
y_pred_train = grid_lgbm_model.predict(X_train)
y_pred_val = grid_lgbm_model.predict(X_val)
y_pred_test = grid_lgbm_model.predict(X_test)

# RMSE
rmse_train = round(np.sqrt(mean_squared_error(y_train, y_pred_train)))
rmse_val = round(np.sqrt(mean_squared_error(y_val, y_pred_val)))
rmse_test =  round(np.sqrt(mean_squared_error(y_test, y_pred_test)))

# Dummy
dummy = DummyRegressor(strategy='mean')
dummy.fit(X_train_selected, y_train)
rmse_dummy = round(np.sqrt(mean_squared_error(y_test, dummy.predict(X_test_selected))))

# Таблица
results = pd.DataFrame({
    'Выборка': ['Train', 'Valid', 'Test', 'Разница Train=>Test', 'Разница Train=>Test (%)'],
    'LGBM': [rmse_train, rmse_val, rmse_test, f'{rmse_test - rmse_train:+.2f}€', f'{(rmse_test - rmse_train) / rmse_train * 100:+.2f}%'],
    'Dummy': [rmse_dummy, rmse_dummy, rmse_dummy, 0, 0],

})

# Транспонируем таблицу
transposed_results = results.T

# Названия строк становятся названиями столбцов
new_header = transposed_results.iloc[0]
transposed_results.columns = new_header
transposed_results.drop('Выборка', axis=0, inplace=True)

# Вывод результата округленного до двух знаков после запятой
print(transposed_results.round(2))

# **6 Выводы**

## **6.1 Основные выводы**

## **6.2. Общее заключение**

В рамках проекта была разработана модель для определения рыночной стоимости автомобилей с пробегом для сервиса «Не бит, не крашен». 

Перед нами стояла задача обучить две модели с метриками не более 2500 € по RMSE. 

Работа выполнена в соответствии с требованиями заказчика по качеству предсказания, скорости работы и времени обучения. Поставленная задача успешно выполнена.


### **6.2.1. Выполненные этапы:**

1. Подготовка данных:

- Загрузка датасета из 354 тыс. записей с 15 признаками

- Разделение на 

    -- 60%: обучающую ( 212 620 строк);

    -- 15%: валидационную (53 156) выборку.

    -- 25%: тестовую (88 593 строк); 

 

- Создание и предварительная обработка в автоматизированном пайплайне из 7 последовательных шагов:

- Коррекция ошибок (замена некорректного значения 0 в `RegistrationMonth` на медиану 6; замена `gasoline` на `petrol`);

- Удаление неинформативных столбцов (`DateCrawled`, `DateCreated`, `LastSeen`, `NumberOfPictures`);

- Проверка и Нормализация дробных чисел;

- Оценка выбросов методом IQR, обработка выбросов методом винзоризации с удалением экстремальных значений (определены и оставлены только нормальные значения для `Power`: 1-1900 л.с., `RegistrationYear`: 1920-2016, `Price` > 1€);

- Обработка пропущенных значений (подстановка `Unknown`);

- Удаление явных дубликатов;

- Проверка на наличие неявных дубликатов и проверка уникальных значений.

Качественная предобработка данных принесла наибольший эффект для качественного обучения модели.

Предобработка проводилась избирательно: все методы применялись к тренировочной выборке и только половина из них к тестовой и валидационной выборке. Выбросы удалялись в тестовой выборке только по цене, так как на проде целевая перемення на вход не приходит, ее надо предсказывать. 

### **6.2.2. Глубокий анализ данных (EDA)**

Ключевой особенностью проекта стал не просто механический подход к обучению моделей, а комплексный исследовательский анализ.

- **Корреляционный анализ:** выявлены наиьболее влиятельные признаки, влияющие на цену автомобиля, устраненеа мультиколлинеарность с коэффициентом более 0.9. Все признаки с коэффициентом  менее 0.9 оказались важны, исключили дополнительно признак `PostalCode`.

- **Поиск скрытых паттернов и взаимосвязей:**

- Анализ однородности дисперсий: проведены тесты Левена, Бартлетта и Флигнера для обнаружения подозрительных паттернов и неоднородности данных

- Проверка парадокса Симпсона: исключена возможность обманчивых корреляций из-за скрытых группировок

- Комплексный анализ неоднородности:

- Тесты однородности дисперсий

- Анализ группировок по категориальным признакам

- Проверка стабильности корреляций

- Поиск скрытых кластеров

- Анализ нелинейных связей

Поиск скрытых взаимосвязей ожидаемой пользы для качества обучения модели не принес, но мы предприняли попытку обнаружить нелинейные скрытые взаимосвязи - таковых необнаружено.

### **6.2.3. Feature Engineering**

- Создали новые признаки: Возраст авто `'age'` вместо пробега `'Kilometer'` (лучше коррелирует), `'region'` на основе `'postal_code'`.

- Отбор признаков с удалением мультиколлинеарных переменных

- Классификация признаков по типу (числовые/категориальные)

- Определение оптимального набора признаков для обучения

- Попытки создания новых признаков путем сложения, перемножения и кластеризации, но  значительного улучшения не принесли)

- Кластеризация данных и создания нового признака на основе кластеризации (ожидаемого эффекта кластеризация не принесла, да и модель LGBM сама внутри это делает)

- При обучении в пайплайн зашщили `SelectFromModel`, которая была призвана отобрать лучшие признаки

### **6.2.4. Обучение и выбор моделей**

Протестированы две модели с использованием разработанного пайплайна:

**1) Лучшей моделью мы определили LightGBM** на тесте

**Модель: LightGBM** ⭐

**RMSE:** 1481 €

**Время обучения:** 6 сек

**Лучшие параметры:** learning_rate=0.11, max_depth=13, n_estimators=250

Проведена гиперпараметрическая оптимизация с использованием GridSearchCV и кросс-валидацией.

**2) Вторая модель Ridge Regression** показала худшие результаты на валидационной выборке:

**RMSE:** 2270 €

**Время обучения:** 6 сек

**Лучшие параметры**: fit_intercept=True, positive=False

✅ Хотя обе модели выполнили требование RMSE < 2500 €

✅ LightGBM показала лучшие результаты по всем критериям:

- На 34% точнее Linear Regression (1466 vs 2225 €)

- На 0.22 сек быстрее в обучении (6.12 vs 6.35 сек)

Оптимальный баланс качества и скорости

### **6.2.5. Оптимизация обученных моделей**

Предприняли попытку извлечения лучших признков, но это оказалось ни к чему, так как модель уже достигла максимальных результатов, поэтому переобучать модель не стали.

### **6.2.6. Ключевые факторы успеха**


**Таких высоких показателей удалось добиться благодаря в первую очередь:**

- Качественной предобработке данных: прежде всего, тщательная обработка выбросов и пропущенных значений, замена ошибочных значений, ограничение максимальных и минимальных значений;

- Подбор наиболее удачных вариантов обработки выбросов и пропусков;

- Подбор гиперпараметров;

- Глубокий EDA: выявление скрытых паттернов, анализ неоднородности, проверка статистических гипотез;

- Систематический подход: создание воспроизводимого пайплайна с автоматической обработкой данных;

- Комплексный анализ: не ограничились базовыми методами, провели многоуровневое исследование структуры данных.

- Обучение нескольких моделей, выбирая лучшую;

**Практическая ценность:**

Разработанное решение обеспечивает:

- Автоматическую обработку новых данных через пайплайн

- Воспроизводимость результатов

- Масштабируемость для больших объемов данных

- Готовность к продакшену: модель готова к интеграции в приложение сервиса

- Модель LightGBM рекомендуется к внедрению для быстрой и точной оценки рыночной стоимости автомобилей в приложении «Не бит, не крашен».